In [61]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
import eli5
from eli5.sklearn import PermutationImportance
from eli5 import explain_prediction
import shap
from sklearn.metrics import precision_score, roc_curve, recall_score, f1_score, roc_auc_score, accuracy_score
from sklearn import metrics
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from hhh import summary_legacy
import warnings
warnings.filterwarnings("ignore")

In [62]:
#数据预处理
def dataprocessing(data):
    dummies_Sex = pd.get_dummies(data['Sex'], prefix='Sex')
    dummies_Sex = pd.DataFrame(dummies_Sex)
    
    ECOG = {'ECOG 0': 0, 'ECOG 1': 1, 'ECOG 2': 2, 'ECOG 3': 3, 'ECOG 4': 4}
    data["ECOG_Performance"] = data["ECOG_Performance"].map(ECOG)
    
    smoker = {'Non-smoker': 0, 'Ex-smoker': 1, 'Current': 2}
    data["Smoking"] = data["Smoking"].map(smoker)
    
    drinker = {'non-drinker': 0, 'light': 1, 'Moderate': 2, 'ex-drinker': 3, 'heavy': 4, 'unknown': 5}
    data["Drinking"] = data["Drinking"].map(drinker)
    
    sub = {'post wall': 1, 'Tonsillar Fossa': 2, 'Base of Tongue': 3, 'Tonsil ': 4,
           'Tonsil Pillar ': 5, 'Soft Palate ': 6, 'Vallecula': 7, 'lat wall': 0}
    data["Subsite"] = data["Subsite"].map(sub)
    
    T = {'T1': 1, 'T2': 2, 'T3': 3, 'T3 (2) ': 4, 'T4a': 5, 'T4b': 0}
    data["T"] = data["T"].map(T)
    
    N = {'N0': 0, 'N1': 1, 'N2a': 2, 'N2b': 3, 'N2c': 4, 'N3': 5}
    data["N"] = data["N"].map(N)
    
    M = {'M0': 0}
    data["M"] = data["M"].map(M)
    
    stage = {'I': 0, 'II': 1, 'III': 2, 'IVA': 3, 'IVB': 4}
    data["Stage"] = data["Stage"].map(stage)
    
    Hpv = {'  Negative': 0, '  positive': 1}
    data["HPVp16status"] = data["HPVp16status"].map(Hpv)
    
    Che = {'none': 0, 'Yes': 1}
    data["Chemotherapy"] = data["Chemotherapy"].map(Che)
    
    RT = {'IMRT': 0, 'IMRT-ipsilat': 1}
    data["RT_Tech"] = data["RT_Tech"].map(RT)
    
    Status = {'Dead': 1, 'Alive': 0}
    data["Status"] = data["Status"].map(Status)
    
    data.drop(['PatientID', 'Sex','Ds_Site', 'Path', 'Primary_Treatment', 'Cause_of_Death',
               'Local_Failure', 'local_failure(days)', 'Regional_Failure', 'regional_failure(days)',
               'Distant_Failure', 'distant_failure (days)', 'Second_Primary', '2nd_cancer_(days)',
               'PMID', 'diagnostics_Versions_PyRadiomics', 'diagnostics_Versions_Numpy',
               'diagnostics_Versions_SimpleITK', 'diagnostics_Versions_PyWavelet', 'diagnostics_Versions_Python',
               'diagnostics_Configuration_Settings', 'diagnostics_Configuration_EnabledImageTypes',
               'diagnostics_Image-original_Hash', 'diagnostics_Image-original_Dimensionality',
               'diagnostics_Image-original_Spacing', 'diagnostics_Image-original_Size',
               'diagnostics_Image-original_Mean',
               'diagnostics_Image-original_Minimum', 'diagnostics_Image-original_Maximum',
               'diagnostics_Mask-original_Hash', 'diagnostics_Mask-original_Spacing',
               'diagnostics_Mask-original_Size',
               'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_VoxelNum',
               'diagnostics_Mask-original_VolumeNum', 'diagnostics_Mask-original_CenterOfMassIndex',
               'diagnostics_Mask-original_CenterOfMass'

               ], axis=1, inplace=True)
    
    data_df=pd.concat([data,dummies_Sex],axis=1)
    return data_df

In [63]:
#处理训练数据
train_data = 'train_data.csv'
data_train = pd.read_csv(train_data)
dataprocessing(data_train)
data_train = data_train.fillna(0)

#标准化
Train_X = data_train.drop(['FU','Status'],axis=1)
#Train_X = data_train.drop(['Status'],axis=1)
Train_y = data_train.pop('FU').values
colNames = Train_X.columns

stdScale = StandardScaler().fit(Train_X) ## 生成规则
Train_X = stdScale.transform(Train_X) ## 将规则应用于训练集

Train_X=pd.DataFrame(Train_X)
Train_X.columns=colNames
# print(z_status)
X_train_lifetime, X_test_lifetime, y_train_lifetime, y_test_lifetime = train_test_split(Train_X, Train_y, test_size=0.3)
# X_train_status, X_test_status, y_train_status, y_test_status = train_test_split(X_status, y_status, test_size=0.3)
# print(Train_X)
shit = pd.DataFrame()
fuck =Train_X.columns
for x in fuck:
    shit[x]=Train_X[x]
# shit['ECOG_Performance']=Train_X['ECOG_Performance']
# print(shit.head())
# print(Train_X)

In [64]:
#训练XGboost
#回归
model_xgb = xgb.XGBRegressor(max_depth=3, learning_rate=0.1, n_estimators=100)
model_xgb.fit(X_train_lifetime, y_train_lifetime)

preds_test = model_xgb.predict(X_test_lifetime)
#分类
# model_xgb =  xgb.XGBClassifier()
# model_xgb.fit(X_train_status, y_train_status)

# train_pre_status = model_xgb.predict(X_train_status)
# test_pre_status = model_xgb.predict(X_test_status)

# # train_pre_proba = model_xgb.predict_proba(X_train_status)
# test_pre_proba = model_xgb.predict_proba(X_test_status)

# # train_auc = roc_auc_score(y_train_status, train_pre_status)
# test_auc = roc_auc_score(y_test_status, test_pre_status)

#roc曲线
# Xgbc_fpr,Xgbc_tpr,Xgbc_threasholds=roc_curve(y_test_status,test_pre_proba[:,1]) # 计算ROC的值,svm_threasholds为阈值
# plt.title("roc_curve of %s(AUC=%.4f)" %('Xgbc_test',test_auc))
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.plot(Xgbc_fpr,Xgbc_tpr)
# plt.show()


#训练svr
# linear_svr = SVR(kernel='rbf', C=100, gamma=0.0001)
# linear_svr.fit(X_train_lifetime, y_train_lifetime)
# preds_test = linear_svr.predict(X_test_lifetime)
#print(preds_test)
print("--------------------------------")
# print(preds_test)
#回归评估
# print(preds_test)
# print("------------------------------------")
# print(y_test_lifetime)
# print("rScore:")
# print(metrics.r2_score(y_test_lifetime, preds_test))
# print("mse:")
# print(mean_squared_error(y_test_lifetime, preds_test))
# print("mae:")
# print(mean_absolute_error(y_test_lifetime, preds_test))


--------------------------------


In [65]:
#置换特征重要性
# perm = PermutationImportance(model_xgb, random_state=1).fit(X_test_lifetime, y_test_lifetime)
# eli5.show_weights(perm, feature_names=X_test_lifetime.columns.tolist(),top=1600)

In [1]:
#Treeshap解释
shap.initjs()
explainer = shap.TreeExplainer(model_xgb)
shap_values = explainer.shap_values(X_test_lifetime)
print(shap_values.shape)
# shap.plots.force(explainer.expected_value, shap_values[1],X_test_lifetime.columns)
# shap.summary_plot(shap_values, X_test_lifetime,max_display=20)
shap.summary_plot(shap_values,X_test_lifetime.columns,plot_type="bar",max_display=10)
# summary_legacy(shap_values,X_test_lifetime.columns,max_display=10)
#xgb.plot_importance(model_xgb,max_num_features=200)

NameError: name 'shap' is not defined